In [3]:
import os

In [1]:
%pwd

'c:\\Users\\Prachi\\Documents\\Chicken-Disease-Classification-Project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Prachi\\Documents\\Chicken-Disease-Classification-Project'

In [6]:
import tensorflow as tf

In [11]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [13]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [14]:
from pathlib import Path

project_root = Path(r"C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project")

CONFIG_FILE_PATH = project_root / "config" / "config.yaml"
PARAMS_FILE_PATH = project_root / "params.yaml"

print("Config file path:", CONFIG_FILE_PATH)
print("Params file path:", PARAMS_FILE_PATH)

print("Config exists?", CONFIG_FILE_PATH.is_file())
print("Params exists?", PARAMS_FILE_PATH.is_file())


Config file path: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\config\config.yaml
Params file path: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\params.yaml
Config exists? True
Params exists? True


In [15]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root],verbose = True )

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [16]:
from urllib.parse import urlparse

In [20]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss" : self.score[0] ,
                   "accuracy" : self.score[1] }
        save_json(path=Path("save.json"), data=scores)



In [21]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evalution = Evaluation(val_config)
    evalution.evaluation()
    evalution.save_score()

except Exception as e:
    raise e

✅ DEBUG YAML PATH: C:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\config\config.yaml
✅ DEBUG YAML CONTENT: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_url': 'https://github.com/PrachiVedant/datasets/raw/refs/heads/main/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}} <class 'dict'>
✅ DEBUG YAML PATH: C:\Us

c:\Users\Prachi\Documents\Chicken-Disease-Classification-Project\venv\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.2979 - loss: 23.4846
